In [10]:
%load_ext autoreload
%autoreload 2
import spacy
import scispacy
from scispacy.linking import EntityLinker
import en_core_sci_lg
from scify.nlp import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of scify.nlp failed: Traceback (most recent call last):
  File "/Users/markus/opt/anaconda3/envs/markus_nlp/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/markus/opt/anaconda3/envs/markus_nlp/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/Users/markus/opt/anaconda3/envs/markus_nlp/lib/python3.8/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/Users/markus/opt/anaconda3/envs/markus_nlp/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 779, in exec_module
  File "<frozen importlib._bootstrap_external>", line 916, in get_code
  File "<frozen importlib._bootstrap_external>", line 846, in source_to_code
  File "<frozen importlib._b

In [2]:
text = "Spinal and bulbar muscular atrophy (SBMA) is an inherited motor neuron disease caused by the expansion of a polyglutamine tract within the androgen receptor (AR). SBMA can be caused by this easily."

In [3]:
from scispacy.abbreviation import AbbreviationDetector
from spacy.pipeline import merge_entities
from scispacy.linking import EntityLinker

nlp = load_sci_pipe()

In [4]:
#linker_umls = EntityLinker(resolve_abbreviations=True, name="umls")
#nlp.add_pipe(linker_umls)

In [11]:
from textacy.corpus import Corpus
from scify.consts import labels
from spacy.tokens import Doc, Token, Span

for label in labels:
    nlp.vocab.strings.add(label)

reannotate()
gnbr_500 = Corpus(nlp).load(nlp, "../data/experiments/GNBR_chem-gene_annotated_500")

#https://github.com/explosion/spaCy/issues/860
#doc = nlp(text)

In [23]:
from scify.utils.datasets import get_csv

dep_paths = get_csv("../data/experiments/GNBR_chem_gene_good_patterns_20k.csv")

In [34]:
dep_paths[1]

{'dep_path': 'enhances|nsubj|start_entity enhances|nmod|up-regulation up-regulation|nmod|end_entity',
 'occurence_count': '1'}

In [39]:
deps = sorted(dep_paths, key=lambda obj: int(obj["occurence_count"]), reverse=True)

In [41]:
deps[:3]

[{'dep_path': 'domain|compound|start_entity domain|nmod|end_entity',
  'occurence_count': '157'},
 {'dep_path': 'inhibitor|appos|start_entity inhibitor|compound|end_entity',
  'occurence_count': '155'},
 {'dep_path': 'fingers|compound|start_entity fingers|nmod|end_entity',
  'occurence_count': '117'}]

## Get Most Potent Paths


## Apply to GNBR itself and check Syntactic Parsing accuracy of SciSpacy

In [ ]:
from IPython.display import Image
Image('../assets/GNBR_datafile_info.png')

## Precision on GNBR
Should be high since the clusters are annotated by humans ... the syntax parsing might make troubles

In [43]:
#get 10 strongest dep paths
wanted_abstr = [doc for doc in gnbr_500 if doc_has_entity_labels(doc,  [["CHEMICAL", "SIMPLE_CHEMICAL"], ["GENE", "GENE_OR_GENE_PRODUCT"]])]
target_sents =[sent for abstract in wanted_abstr for sent in abstract.sents if doc_has_entity_labels(sent,  [["CHEMICAL", "SIMPLE_CHEMICAL"], ["GENE", "GENE_OR_GENE_PRODUCT"]])]
len(wanted_abstr), len([sent for abstract in wanted_abstr for sent in abstract.sents ]), len(target_sents)

(207, 210, 206)

In [ ]:
from scify.utils.datasets import GNBR
data, distributions = GNBR.get_data_and_distributions("../DATA/biomedrel/")

In [44]:
ents_w_candidates(target_sents[1])

[(c-Myc, ['GENE_OR_GENE_PRODUCT', 'PROTEIN']),
 (10058-F4, ['CHEMICAL', 'SIMPLE_CHEMICAL']),
 (induces, []),
 (cell-cycle arrest, ['DISEASE']),
 (apoptosis, []),
 (myeloid, ['CELL']),
 (human, ['TAXON', 'ORGANISM']),
 (acute_myeloid_leukemia, ['DISEASE'])]

In [22]:
def get_range(coll):
    return (min(coll), max(coll))

In [ ]:
#chem-gene or gene-chem
def match_links_entities(doc, subtree, start_entity=["CHEMICAL", "SIMPLE_CHEMICAL"], end_entity=["GENE", "GENE_OR_GENE_PRODUCT"])->bool:
    """checks if match is a wanted pair. TODO: let construct_pattern handle this from the start
    subtree ~ [0, 5, 8, ...] (tokenIdx)
    """
    

In [ ]:
from typing import Tuple
def theme_support(dep_path, dist:Dict)->List[Tuple]:
    """get non-null GNBR supports for a given dependency path (needs distributions)"""
    supports = dist[dep_path].items()
    return [(a,b) for a,b in supports if b>0]

In [51]:
type(target_sents[1].text)

str

In [61]:
example = [
    "prevented|nsubj|START_ENTITY prevented|dobj|END_ENTITY",
    "causes|nsubj|START_ENTITY causes|dobj|END_ENTITY"
]
patterns = [dep["dep_path"] for dep in deps[:15]]
patterns

['domain|compound|start_entity domain|nmod|end_entity',
 'inhibitor|appos|start_entity inhibitor|compound|end_entity',
 'fingers|compound|start_entity fingers|nmod|end_entity',
 'protein|compound|start_entity protein|nsubj|end_entity',
 'factor|compound|start_entity factor|nsubj|end_entity',
 'finger|compound|start_entity finger|nmod|end_entity',
 'protein|compound|start_entity protein|appos|end_entity',
 'gene|compound|start_entity gene|appos|end_entity',
 'dismutase|compound|start_entity dismutase|appos|end_entity',
 'finger|compound|start_entity finger|compound|end_entity',
 'inhibitor|nsubj|start_entity inhibitor|nmod|end_entity',
 'factor|compound|start_entity factor|appos|end_entity',
 'region|compound|start_entity region|nmod|end_entity',
 'finger|compound|start_entity finger|appos|end_entity',
 'transporter|compound|start_entity transporter|appos|end_entity']

In [63]:
["a", "b"] + ["c"]

['a', 'b', 'c']

In [67]:
matcher = add_matches(nlp.vocab, patterns, start_ents=["CHEMICAL", "SIMPLE_CHEMICAL"], end_ents=["GENE", "GENE_OR_GENE_PRODUCT"])
matched_sents = match_texts(matcher, [nlp(ts.text) for ts in target_sents] ,nlp)
matched_sents

{'inhibitor|appos|start_entity inhibitor|compound|end_entity': [{'doc_idx': 205,
   'span': 'c-Myc inhibitor , 10058-F4',
   'sents': [A small-molecule c-Myc inhibitor , 10058-F4 , induces cell-cycle arrest , apoptosis , and myeloid differentiation of human acute_myeloid_leukemia .],
   'matches': [[3, 5, 2]],
   'sent_ents': [[small-molecule,
     inhibitor,
     10058-F4,
     induces cell-cycle arrest,
     apoptosis,
     myeloid,
     differentiation,
     human]]},
  {'doc_idx': 205,
   'span': 'small molecule c-Myc inhibitor , 10058-F4',
   'sents': [Furthermore , we found a comparable decrease in proliferation and G0/G1 accumulation of 786-0 and RC-2 cells after treatment with a small molecule c-Myc inhibitor , 10058-F4 .],
   'matches': [[23, 25, 22]],
   'sent_ents': [[decrease,
     proliferation,
     G0/G1,
     accumulation,
     RC-2 cells,
     treatment,
     small molecule c-Myc,
     10058-F4]]},
  {'doc_idx': 205,
   'span': 'PTEN inhibitor , potassium_bisperoxo',
 

### What pattern yields most on PubMed? On GNBR_500 (might be sorted...and skewed sentences)

## Precision on PubMed
Get pubmed sentences with Chem-Gene pairs (500 in the 750 abstracts) and see if there's some matches with the top dep patterns from GNBR

In [ ]:
#TODO check if theme support among paths is evenly distributed

In [ ]:
#invalid pattern
construct_pattern('causes|nsubj|END_ENTITY causes|dobj|START_ENTITY')